In [ ]:
import os
import re

import pandas as pd
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
%matplotlib notebook

from parsers import generic, dlaf, slate, dplasma, scalapack

colors_default = plt.cm.get_cmap('tab10').colors

LIB_COLOR = {
    'dlaf': colors_default[2], 
    'slate': colors_default[0],
    'dplasma': colors_default[1],
    'scalapack': colors_default[4],
}

## Collect data from filesystem

In [ ]:
slurm_db = generic.parse_slurm_db('slurm.db')

In [ ]:
campaigns = {
    'pre': 'raw_data/pre',
    'post': 'raw_data/post/trsm',
}

jobs_reports = []

df = pd.DataFrame([], columns=[
    'library',
    'benchmark_lib',
    'run_index',
    'm',
    'n',
    'mb',
    'grid_rows',
    'grid_cols',
    'time',
    'performance',
    'id',
    'campaign',
])

for campaign_name, workdir in campaigns.items():
    for entry in os.listdir(workdir):
        m = re.match('(\d+)J\d+_\d+', entry)

        if not m:
            continue

        jobid, = m.groups()
        jobdata = slurm_db[jobid]
        jobname = jobdata['jobname']

        first_seen = jobid not in jobs_reports
        jobs_reports.append(jobid)

        if jobdata['state'] not in ['COMPLETED', 'TIMEOUT']:
            if first_seen:
                display(f'WARNING {jobid} {jobname} {jobdata["state"]}')
            continue

        # select parser
        if re.search(r'.*slate.*', jobname):
            benchmark_lib = 'slate'
            parser = slate.parse_trsm
        elif re.search(r'.*dplasma.*', jobname):
            benchmark_lib = 'dplasma'
            parser = dplasma.parse_trsm
        elif re.search(r'.*scalapack.*', jobname):
            benchmark_lib = 'scalapack'
            parser = scalapack.parse_trsm
        else:
            benchmark_lib = 'dlaf'
            parser = dlaf.parse_trsm

        current_run = os.path.join(workdir, entry)
        for task_entry in os.listdir(current_run):
            if not task_entry.endswith('.out'):
                continue

            out_filepath = os.path.join(current_run, task_entry)
            try:
                current_data = parser(out_filepath)
            except Exception as e:
                display(f'ERROR {out_filepath} {e}')
                continue

            if len(current_data[1]) == 0:
                display(f'No data in {out_filepath}')
                continue

            current_df = pd.DataFrame(current_data[1], columns=[
                'run_index',
                'm',
                'n',
                'mb',
                'grid_rows',
                'grid_cols',
                'time',
                'performance',
            ])

            # do not plot results with more than 64 nodes
            if (jobdata["nodes"] > 64):
                continue

            current_df['campaign'] = campaign_name
            current_df['library'] = benchmark_lib
            current_df['nodes'] = jobdata["nodes"]
            current_df['ranks_per_node'] = current_df['grid_rows'] * current_df['grid_cols'] / current_df["nodes"]
            current_df['benchmark_lib'] = benchmark_lib + "|" + str(int(current_df['ranks_per_node'].iloc[0]))
            current_df['performance_per_node'] = current_df['performance'] / current_df['nodes']

            current_df['id'] = entry

            df = df.append(current_df)

display("LOADED")

In [ ]:
df

In [ ]:
df_runs = df\
.loc[df['run_index'] != 0]\
.groupby(['library', 'm', 'n', 'mb', 'nodes', 'ranks_per_node', 'benchmark_lib', 'campaign'])\
.agg(
    p_mean=("performance", 'mean'),
    p_min=("performance", 'min'),
    p_max=("performance", 'max'),
    ppn_mean=("performance_per_node", 'mean'),
    ppn_min=("performance_per_node", 'min'),
    ppn_max=("performance_per_node", 'max'),
    time_mean=("time", 'mean'),
    time_min=("time", 'min'),
    time_max=("time", 'max'),
    measures=('performance', 'count'),
    ids=('id', 'sum')
)\
.reset_index()

display(df_runs)

In [ ]:
df_groups = df_runs[df_runs['mb'] == 512].groupby(['m', 'n'])

domain_m = df['m'].unique().tolist()
domain_n = df['n'].unique().tolist()

domain_m.sort()
domain_n.sort()

fig, axs_perf = plt.subplots(len(domain_m), 2,
                             squeeze=False, sharey='row', figsize=(9,12))
fig, axs_time = plt.subplots(len(domain_m), 2,
                             squeeze=False, figsize=(9,12))

for (m, n), grp_data in df_groups:
    ax_i, ax_j = domain_m.index(m), 1 if m == n else 0
    ax_perf = axs_perf[ax_i, ax_j]
    ax_time = axs_time[ax_i, ax_j]
    
    for (lib, campaign), lib_data in grp_data.groupby(['benchmark_lib', 'campaign']):
        lib_name, lib_variant = lib.split("|")
        lib_data.plot(ax=ax_perf, x='nodes', y='ppn_mean',
                      marker='.', linestyle = '-' if lib_variant == '1' else '--',
                      label=f'{lib}@{campaign}', color=LIB_COLOR[lib_name])
        lib_data.plot(ax=ax_time, x='nodes', y='time_mean',
                      marker='.', linestyle = '-' if lib_variant == '1' else '--',
                      label=f'{lib}@{campaign}', color=LIB_COLOR[lib_name])

    ax_perf.set_ylabel(f'GFlops/node (with m={m})')
    ax_perf.set_ylim(0, 1200)
    ax_time.set_ylabel(f'Time (s) (with m={m})')
    
    for ax in [ax_perf, ax_time]:
        ax.legend(loc="upper right")
        ax.set_xscale('log', basex=2)
        ax.set_xticks([2**x for x in range(9)])
        ax.set_xlabel(f'nodes (with n={n})')
        
        ax.grid(color='gray', alpha=0.3)

In [ ]:
df_runs_multi = df_runs.set_index(['library', 'ranks_per_node', 'mb', 'campaign']).sort_index()

display(df_runs_multi)

In [ ]:
def plot_interesting(interesting_configs, plot_time=False):
    if type(interesting_configs) == list:
        interesting_configs = {k:{} for k in interesting_configs}
    
    domain_m = df['m'].unique().tolist()
    domain_n = df['n'].unique().tolist()

    domain_m.sort()
    domain_n.sort()

    fig, axs_perf = plt.subplots(len(domain_m), 2,
                                 squeeze=False, sharey='row', figsize=(9, 12))

    if plot_time:
        fig, axs_time = plt.subplots(len(domain_m), 2,
                                     squeeze=False, figsize=(9, 12))

    
    for config in interesting_configs.keys():
        plot_properties = {
            # default properties
            **dict(color=LIB_COLOR[config[0]], marker='.', linestyle = '-'),
            # passed-by-argument
            **interesting_configs[config]
        }
        for (m, n), grp_data in df_runs_multi.loc[config, :].groupby(['m', 'n']):
            ax_i, ax_j = domain_m.index(m), 1 if m == n else 0
            ax_perf = axs_perf[ax_i, ax_j]
            
            if plot_time:
                ax_time = axs_time[ax_i, ax_j]

            for (lib, campaign_name), lib_data in grp_data.groupby(['benchmark_lib', 'campaign']):
                lib_name, lib_variant = lib.split("|")
                lib_data.plot(ax=ax_perf, x='nodes', y='ppn_mean',
                              label=config if len(config) > 3 else f'{config} {campaign_name}',
                              **plot_properties)
                ax_perf.fill_between(lib_data['nodes'], lib_data['ppn_min'], lib_data['ppn_max'],
                                     alpha=0.2, color=plot_properties['color'])
                
                if plot_time:
                    lib_data.plot(ax=ax_time, x='nodes', y='time_mean',
                                label=config if len(config) > 3 else f'{config} {campaign_name}',
                                 **plot_properties)
                    
            axs = [ax_perf]
            
            ax_perf.set_ylabel(f'GFlops/node\n(with m={m})')
            ax_perf.set_ylim(0, 1200)

            if plot_time:
                axs.append(ax_time)
                if ax_j == 0:
                    ax_time.set_ylabel(f'Time (s)\n(with m={m})')
                
            for ax in axs:
                ax.legend(loc="upper right")
                ax.set_xscale('log', basex=2)
                ax.set_xticks([2**x for x in range(7)])
                ax.set_xlabel(f'nodes (with n={n})')
                ax.grid(color='gray', alpha=0.3)

In [ ]:
def plot_interesting_pres(m, n, interesting_configs, plot_time=False):
    if type(interesting_configs) == list:
        interesting_configs = {k:{} for k in interesting_configs}
    
    fig, axs_perf = plt.subplots(1, 1,
                                 squeeze=False, sharey='row', figsize=(4.5, 3.8))

    if plot_time:
        fig, axs_time = plt.subplots(1, 1,
                                     squeeze=False, figsize=(4.5, 3.8))

    
    for config in interesting_configs.keys():
        plot_properties = {
            # default properties
            **dict(color=LIB_COLOR[config[0]], marker='.', linestyle = '-'),
            # passed-by-argument
            **interesting_configs[config]
        }
        for (current_m, current_n), grp_data in df_runs_multi.loc[config, :].groupby(['m', 'n']):
            if current_m != m or current_n != n:
                continue
            
            ax_i, ax_j = 0, 0
            ax_perf = axs_perf[ax_i, ax_j]
            
            if plot_time:
                ax_time = axs_time[ax_i, ax_j]

            for (lib, campaign_name), lib_data in grp_data.groupby(['benchmark_lib', 'campaign']):
                lib_name, lib_variant = lib.split("|")
                lib_data.plot(ax=ax_perf, x='nodes', y='ppn_mean', **plot_properties)
                ax_perf.fill_between(lib_data['nodes'], lib_data['ppn_min'], lib_data['ppn_max'],
                                     alpha=0.2, color=plot_properties['color'])
                
                if plot_time:
                    lib_data.plot(ax=ax_time, x='nodes', y='time_mean', **plot_properties)
                    
            axs = [ax_perf]
            
            ax_perf.set_ylabel(f'GFlops/node')
            ax_perf.set_ylim(0, 1200)

            if plot_time:
                axs.append(ax_time)
                if ax_j == 0:
                    ax_time.set_ylabel(f'Time (s)')
                
            for ax in axs:
                ax.legend(loc="upper right")
                ax.set_xscale('log', basex=2)
                ax.set_xticks([2**x for x in range(7)])
                ax.set_xlabel(f'nodes')
                ax.grid(color='gray', alpha=0.3)
                ax.set_title(f'Matrix Size {m}x{n}')

In [ ]:
plot_interesting_pres(40960, 20480, {
    ('dlaf', 2, 512, 'post'): dict(label='DLAF'),
    ('slate', 2, 512, 'post'): dict(label='SLATE'),
    ('dplasma', 1, 384): dict(label='DPlasma'),
    ('scalapack', 36, 192): dict(label='ScaLAPACK (libsci)'),
}, True)

In [ ]:
plot_interesting_pres(40960, 40960, {
    ('dlaf', 2, 512, 'post'): dict(label='DLAF'),
    ('slate', 2, 512, 'post'): dict(label='SLATE'),
    ('dplasma', 1, 384): dict(label='DPlasma'),
    ('scalapack', 36, 192): dict(label='ScaLAPACK (libsci)'),
}, True)

In [ ]:
#df_runs_multi[(df_runs_multi.benchmark_lib == 'slate|2')& (df_runs_multi.nodes == 1)]

In [ ]:
plot_interesting({
    ('dlaf', 1, 512, 'post'): {},
    ('dlaf', 2, 512, 'post'): dict(linestyle='--'),
}, True)

In [ ]:
plot_interesting({
    ('slate', 1, 512, 'post'): {},
    ('slate', 2, 512, 'post'): dict(linestyle='--'),
}, True)

In [ ]:
for (campaign, mb) in [(campaign, mb) for campaign in campaigns.keys() for mb in [256, 512]]:
    plot_interesting([
        ('dlaf', 2, mb, campaign),
        ('slate', 2, mb, campaign),
        ('dplasma', 1, mb, campaign),
        ('scalapack', 36, 192, 'post'),
    ], True)

### No Regression

In [ ]:
for mb in [256, 512]:
    plot_interesting({
        ('dlaf', 2, mb, 'pre'): dict(color='black', linestyle='--'),
        ('dlaf', 2, mb, 'post'): {},
    }, True)

In [ ]:
plot_interesting({
    ('dlaf', 1, 256, 'post'): dict(color='orange', linestyle='--', alpha=0.3),
    ('dlaf', 2, 256, 'post'): dict(color='red'),
    ('dlaf', 1, 512, 'post'): dict(color='cyan', linestyle='--', alpha=0.3),
    ('dlaf', 2, 512, 'post'): dict(color='green'),
}, True)

In [ ]:
plot_interesting({
    ('dplasma', 1, 192, 'post'): dict(color='red'),
    ('dplasma', 1, 256, 'post'): dict(color='orange'),
    ('dplasma', 1, 288, 'post'): dict(color='yellow'),
    ('dplasma', 1, 384, 'post'): dict(color='green'),
    ('dplasma', 1, 512, 'post'): dict(color='cyan'),
})

In [ ]:
for rank_per_node in [1,2]:
    plot_interesting({
        ('slate', rank_per_node, 192, 'post'): dict(color='red'),
        ('slate', rank_per_node, 256, 'post'): dict(color='orange'),
        ('slate', rank_per_node, 288, 'post'): dict(color='yellow'),
        ('slate', rank_per_node, 384, 'post'): dict(color='green'),
        ('slate', rank_per_node, 512, 'post'): dict(color='cyan'),
    }, True)

In [ ]:
plot_interesting({
    ('scalapack', 36, 64, 'post'): dict(color='red'),
    ('scalapack', 36, 128, 'post'): dict(color='orange'),
    ('scalapack', 36, 192, 'post'): dict(color='yellow'),
    ('scalapack', 36, 256, 'post'): dict(color='green'),
}, True)

In [ ]:
for ranks_per_node in [1,2]:
    plot_interesting({
        ('dlaf', ranks_per_node, 192, 'post'): dict(color='red'),
        ('dlaf', ranks_per_node, 256, 'post'): dict(color='orange'),
        ('dlaf', ranks_per_node, 288, 'post'): dict(color='yellow'),
        ('dlaf', ranks_per_node, 384, 'post'): dict(color='green'),
        ('dlaf', ranks_per_node, 512, 'post'): dict(color='cyan'),
    }, True)

In [ ]:
for campaign in campaigns.keys():
    plot_interesting({
        ('dplasma', 1, 256, campaign): dict(linestyle='--'),
        ('dplasma', 1, 512, campaign): {},
        ('dlaf', 2, 256, campaign): dict(linestyle='--'),
        ('dlaf', 2, 512, campaign): {},
    })